In [17]:
import os, sys, codecs, time, datetime
import fileinput
import os.path
from io import StringIO
from subprocess import call


from docopt import docopt
from propsde.applications.viz_tree import DepTreeVisualizer

import propsde.applications.run as run
import sys
if sys.version_info[0] >= 3:
    unicode = str
    
stanza.download('de')
nlp = stanza.Pipeline('de')

2020-05-11 13:25:59 INFO: Downloading default packages for language: de (German)...
2020-05-11 13:26:00 INFO: File exists: /home/chris/stanza_resources/de/default.zip.
2020-05-11 13:26:04 INFO: Finished downloading models and saved to /home/chris/stanza_resources.
2020-05-11 13:26:04 INFO: Loading these models for language: de (German):
| Processor | Package |
-----------------------
| tokenize  | gsd     |
| mwt       | gsd     |
| pos       | gsd     |
| lemma     | gsd     |
| depparse  | gsd     |
| ner       | conll03 |

2020-05-11 13:26:04 INFO: Use device: gpu
2020-05-11 13:26:04 INFO: Loading: tokenize
2020-05-11 13:26:04 INFO: Loading: mwt
2020-05-11 13:26:04 INFO: Loading: pos
2020-05-11 13:26:05 INFO: Loading: lemma
2020-05-11 13:26:05 INFO: Loading: depparse
2020-05-11 13:26:05 INFO: Loading: ner
2020-05-11 13:26:06 INFO: Done loading processors!


In [2]:
import pickle

In [3]:
with open('polizeiberichte.pkl', 'rb') as f:
    data = pickle.load(f, encoding="utf8")

In [4]:
text_example = data[72]["text1"]
print(text_example)

Der Polizeiliche Staatsschutz beim Landeskriminalamt ermittelt wegen gemeinschädlicher Sachbeschädigung an einem jüdischen Mahnmal in Tiergarten. Passanten hatten heute früh gegen 8.50 Uhr gelbe Schmierereien an der Gedenktafel in der Levetzowstraße entdeckt und die Polizei alarmiert.


In [5]:
text_example = text_example.replace(". ", ".\n")

In [6]:
print(text_example)

Der Polizeiliche Staatsschutz beim Landeskriminalamt ermittelt wegen gemeinschädlicher Sachbeschädigung an einem jüdischen Mahnmal in Tiergarten.
Passanten hatten heute früh gegen 8.50 Uhr gelbe Schmierereien an der Gedenktafel in der Levetzowstraße entdeckt und die Polizei alarmiert.


In [39]:
file = codecs.open("examples.txt", encoding='utf8')
gs = run.parseSentences(file)

running mate-tools
lemmatizing
parsing + tagging
collapsing
java -jar ext/org.jobimtext.collapsing.jar -i tmp/parsed.conll06 -o tmp -sf -l de -r ext/resources/german_modified.txt -f c -np -nt


In [40]:
for g,tree in gs: 
    for prop in g.getPropositions('pdf'):
        curProp = '{0}:({1})'.format(prop.pred,
                                      ", ".join([rel + ":" + arg for rel,arg in sorted(prop.args,key=lambda rel:prop.rel_order(rel[0]))]))
        print(curProp)

fahren:(subj:Der regierende Bürgermeister , prep_zu:Rathaus )
gehen:(subj:er , prep_in:roten Rathaus , prep_in:den Sitzungssaal )
liegen:(subj:Der alte Sitzungssaal , prep_neben:dem Büro von Michael Müller )


In [43]:
prop = g.getPropositions('pdf')[0]
print(prop.args)
print(prop.pred)

[['subj', 'Der alte Sitzungssaal '], ['prep_neben', 'dem Büro von Michael Müller ']]
liegen


In [44]:
test = gs[0][0]
print(test)

Der regierende Bürgermeister fährt zum Rathaus .
20	2,regierende	ADJA	0	0	mod,21
21	3,Bürgermeister	NN	0	0	subj,22
22	4,fährt	VVFIN	1	1	
23	6,Rathaus	NN	0	0	prep_zu,22



In [58]:
from itertools import compress
import stanza

def _map_article_of_subj_to_pronoun(arg):
    personal_pronouns = {"der": "er", "das": "es", "die": "sie", "dem": "er", "den": "sie"}
    
    if "subj" in arg:
        words = arg[1].lower().split()
        pronoun_place = list(word in personal_pronouns for word in words)
        pronoun = list(compress(words, pronoun_place))
        if len(pronoun) == 1:
            current_subj = (personal_pronouns[pronoun[0]], arg[1])
            return current_subj
    return None

def _get_core_from_noun_phrase(noun_phrase):
    pos_tag = nlp(noun_phrase)
    for sent in pos_tag.sentences:
        for word in sent.words:
            if word.xpos == "NN" or word.xpos == "NE":
                return word.lemma
    


In [175]:
subjects = []
nouns = []
prop_cache = []
index = 0

for i, (g, tree) in enumerate(gs):
    subjects.append([])
    for prop in g.getPropositions('pdf'):
        prop_cache.append({"args": prop.args, "pred": prop.pred})
        
while index < len(prop_cache):
    for i, arg in enumerate(prop_cache[index]["args"]):
        words = arg[1].lower().split()
        _article_pronoun = _map_article_of_subj_to_pronoun(arg)
        _noun_phrase_core = _get_core_from_noun_phrase(arg[1])
        if _article_pronoun:
            subjects[i].append(_article_pronoun)
        if _noun_phrase_core:
            nouns.append(_noun_phrase_core)
            nouns = list(set(nouns))
        
        if _noun_phrase_core in nouns:
            #prop_cache[index]["args"][i][1] = {"core": _noun_phrase_core, "np": arg[1]}
            prop_cache[index]["args"][i][1] = _noun_phrase_core
        if i > 0 and len(subjects[i-1]) > 0 and subjects[i-1][0][0] in words:
            prop_cache[index]["args"][i][1] = _get_core_from_noun_phrase(subjects[i-1][0][1])
    

    index += 1
    

Der regierende Bürgermeister fährt zum Rathaus.
Im roten Rathaus geht er in den Sitzungssaal.
Der alte Sitzungssaal liegt neben dem Büro von Michael Müller.

In [176]:
for prop in prop_cache:
    print(prop)

{'args': [['subj', 'Bürgermeister'], ['prep_zu', 'Rathaus']], 'pred': 'fahren'}
{'args': [['prep_in', 'Rathaus'], ['subj', 'Bürgermeister'], ['prep_in', 'Sitzungssaal']], 'pred': 'gehen'}
{'args': [['subj', 'Sitzungssaal'], ['prep_neben', 'Büro']], 'pred': 'liegen'}
